# Head Files

In [1]:
import torch
import torchvision
# from torch.utils.tensorboard import SummaryWriter
import time
from torch import nn
from torch.utils.data import DataLoader
from load_data import MyData  # self-made
from torchvision import transforms
from tqdm import tqdm_notebook as tqdm # View procedure
import os
import scipy.io
from random import random
import numpy as np
import gc
from torch.utils.tensorboard import SummaryWriter
from net_cnn_lstm1 import MyNetwork
from torchnlp.word_to_vector import GloVe
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

# 训练模型

## 设置超参数

In [2]:
BATCH_SIZE = 1
C,H,W = 1,1,2400
learn_rate = 0.0005
num_epochs = 50

## 设置随机种子

In [3]:
import random
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
manualSeed = 32
random.seed(manualSeed)
torch.manual_seed(manualSeed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(manualSeed)

## 设置优化器和损失函数

In [4]:
import torch.optim as optim
device = torch.device("cuda:0")
gc.collect()
torch.cuda.empty_cache()

# ==损失函数权重
# ======== 二分类HC/DOC
# 计算总样本数量
# condition1
# total_samples = 887 + 985 + 879
# condition2
# total_samples = 929 + 1029 + 886
# condition3
# total_samples = 887 + 975 + 879
# rest
total_samples = 852 + 1051 + 872
# 计算每个类别的权重
# condition1
# weights = [total_samples / 887, total_samples / (985 + 879)]
# condition2
# weights = [total_samples / 929, total_samples / (1029 + 886)]
# condition3
weights = [total_samples / 852, total_samples / (1051 + 872)]

# ======== 二分类MCS/UWS
# 计算总样本数量
# condition1
# total_samples = 985 + 879
# condition2
# total_samples = 1029 + 886
# condition3
# total_samples = 975 + 879
# rest
# total_samples = 1051 + 872
# 计算每个类别的权重
# condition1
# weights = [total_samples / 985, total_samples / 879]
# condition2
# weights = [total_samples / 1029, total_samples / 886]
# condition3
# weights = [total_samples / 975, total_samples / 879]
# condition3
# weights = [total_samples / 1051, total_samples / 872]
# 将权重转换为张量
weights_tensor = torch.tensor(weights, device=device)

# 定义交叉熵损失函数并设置权重
criterion = nn.CrossEntropyLoss(weight=weights_tensor)
# criterion = nn.CrossEntropyLoss()
criterion = criterion.to(device)

## Training

In [ ]:
def make_dataset(dataset,mode):
    # find the fold file
    count = 0
    for person in range(len(dataset)):
        filename = os.path.join(dataset.path, dataset.file_path[person])
        # extract the pure name of the file
        parts = filename.split("\\")
        file_name = parts[-1]
        name_without_extension = file_name.split(".")[0]
        # label or data
        file_last = name_without_extension.split("_")[-1]
        if file_last.isdigit(): # data
            # is this fold or not
            if int(file_last) == fold: # yes
                print(filename)
                count = count + 1
                data_map = torch.load(filename)
                # train or test
                if name_without_extension.split("_")[-2] == "train":
                    for i in range(data_map.size(0)):
                        train_data.append(data_map[i])
                elif name_without_extension.split("_")[-2] == "test":
                    for i in range(data_map.size(0)):
                        test_data.append(data_map[i])
                if count == 4:
                    del data_map
                    gc.collect()
                    torch.cuda.empty_cache() 
                    break
            else:   # not
                pass
        else: # label
            file_last = name_without_extension.split("_")[-4]
            file_mode = name_without_extension.split("_")[-2]
            if file_mode == mode.split("_")[-1]: # is this mode or not
                # is this fold or not
                if int(file_last) == fold: # yes
                    print(filename)
                    count = count + 1
                    data_map = torch.load(filename)
                    # train or test
                    if name_without_extension.split("_")[-5] == "train":
                        for i in range(data_map.size(0)):
                            train_label.append(data_map[i])
                    elif name_without_extension.split("_")[-5] == "test":
                        for i in range(data_map.size(0)):
                            test_label.append(data_map[i])
                    if count == 4:
                        del data_map
                        gc.collect()
                        torch.cuda.empty_cache() 
                        break
                else:   # not
                    pass
            else:
                pass
        del filename, parts, file_name, name_without_extension, file_last
        gc.collect()
        torch.cuda.empty_cache()    
# 定义LSTM超参数
input_size = 64  # 输入特征维度
hidden_size = 64  # 隐藏单元数量
num_layers = 2  # LSTM层数
output_size = 2  # 输出类别数量
# 创建模型实例
# model_list = ['', '_CNN', '_CNN_spa', '_CNN_spa_lstm']
# model_name = model_list[0]
# if model_name == model_list[0]: #  CascadeCept
#     from network_cnn_lstm import MyNetwork
# elif model_name == model_list[1]: #  CNN
#     from network_cnn_lstm_2 import MyNetwork
# elif model_name == model_list[2]: # CascadeCept_1
#     from network_cnn_lstm_3 import MyNetwork
# elif model_name == model_list[3]: # CascadeCept_2
#     from network_cnn_lstm_4 import MyNetwork
model_list = ['_1', '_2', '_3', '_4']
model_name = model_list[1]
if model_name == model_list[0]: #  CascadeCept
    from net_cnn_lstm1 import MyNetwork
elif model_name == model_list[1]: #  CascadeCept:
    from net_cnn_lstm2 import MyNetwork
model = MyNetwork(input_size, hidden_size, num_layers, output_size)
model = model.to(device)

# experimental dir: rest, conditionA, conditionB, conditionC
exper_dir = "rest"
root_dir = f"../data/eegmap_split/{exper_dir}"
# classification = "hc_doc"/doc or "mcs_uws"/uws
classification = "hc_doc"
# classification = "mcs_uws"
fold_num = 5
for fold in tqdm(range(5)):
    # train num folds
#     fold = 0 # 选择折数
    # -- prepare datasets
    train_data = []
    train_label = []
    test_data = []
    test_label = []
    
    if classification == "hc_doc":
        # ---- hc
        dataset = MyData(root_dir, "train", "hc") # hc
        make_dataset(dataset,classification)
        dataset = MyData(root_dir, "test", "hc") # hc
        make_dataset(dataset,classification)
    # ---- mcs
    dataset = MyData(root_dir, "train", "mcs") # mcs
    make_dataset(dataset,classification)
    dataset = MyData(root_dir, "test", "mcs") # hc
    make_dataset(dataset,classification)
    # ---- uws
    dataset = MyData(root_dir, "train", "uws") # uws
    make_dataset(dataset,classification)
    dataset = MyData(root_dir, "test", "uws") # hc
    make_dataset(dataset,classification)
    
    print(torch.stack(train_data).size())
    print(torch.stack(train_label).size())
    print(torch.stack(test_data).size())
    print(torch.stack(test_label).size())
    del dataset
    gc.collect()
    torch.cuda.empty_cache()  
    
    train_data = torch.stack(train_data)
    train_label = torch.stack(train_label)
    test_data = torch.stack(test_data)
    test_label = torch.stack(test_label)
    # train dataset
    train_td = TensorDataset(train_data, train_label)
    train_loader = DataLoader(train_td, batch_size = BATCH_SIZE, shuffle = True)
    # test dataset
    test_td = TensorDataset(test_data, test_label)
    test_loader = DataLoader(test_td, batch_size = BATCH_SIZE, shuffle = True)
    del train_data, train_label, test_data, test_label, train_td, test_td
    gc.collect()
    torch.cuda.empty_cache()
    
    # set model for each fold
    model = MyNetwork(input_size, hidden_size, num_layers, output_size)
    model = model.to(device)
    optimizer = optim.Adam(model.parameters(), lr = learn_rate)
    scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[10, 30, 50], gamma=0.2)
    # -- start training
    start_time = time.time()
    # train and test step records
    total_train_step = 0
    total_test_step = 0
    min_test_loss = 1000
    # add Tensorboard
    writer_train = SummaryWriter(f"../logs/{classification}/{exper_dir}{model_name}/logs_train_{fold}")
    writer_valid = SummaryWriter(f"../logs/{classification}/{exper_dir}{model_name}/logs_test_{fold}")
    writer_valid_acc = SummaryWriter(f"../logs/{classification}/{exper_dir}{model_name}/logs_test_acc_{fold}")
    for i in tqdm(range(num_epochs)):  
        print(f"========= Epoch {i} Training =========")
        # train steps
        model.train()
        for data in train_loader:
            # x, y
            data_map, label=data
            data_map_reshaped = torch.reshape(data_map, (110, 1, 1, 2400))
            label_int = label.long()
            data_map_reshaped=data_map_reshaped.to(device)
            label_int=label_int.to(device)
            del data_map, label
            gc.collect()
            torch.cuda.empty_cache()
            # y_pred
            label_pred = model(data_map_reshaped)
            # Loss Computation and Optimization
            loss = criterion(label_pred,label_int)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            # draw tensorboard
            total_train_step = total_train_step + 1
            # print info
            if total_train_step % 1000 == 0:
                end_time = time.time()
                print(label_pred)
                print(f"Train time: {end_time - start_time}")
                print(f"Train steps: {total_train_step}, Loss: {loss.item()}")
            writer_train.add_scalar("train_loss",loss.item(),total_train_step)
            # Clear gpu
            del data, data_map_reshaped, label_int, label_pred, loss
            gc.collect()
            torch.cuda.empty_cache()
        
        # Evaluation and save the best model
        print(f"========= Epoch {i} Testing =========")
        model.eval()
        total_test_loss = 0
        test_count = 0
        total_test_acc = 0
        with torch.no_grad():
            for data in test_loader:
                test_count = test_count + 1
                # x, y
                data_map, label=data
                data_map_reshaped = torch.reshape(data_map, (110, 1, 1, 2400))
                label_int = label.long()
                data_map_reshaped = data_map_reshaped.to(device)
                label_int = label_int.to(device)
                del data_map, label
                gc.collect()
                torch.cuda.empty_cache()
                # y_pred
                label_pred_test = model(data_map_reshaped)
                loss = criterion(label_pred_test,label_int)
#                 print(label_pred_test)
                # accuracy 
                total_test_acc = total_test_acc + ((label_pred_test.argmax(1)) == label_int).sum()
                # draw tensorboad
                total_test_loss = total_test_loss + loss
                if test_count % 100 == 0:
                    print(f"Loss: {total_test_loss} Accuracy: {total_test_acc/test_count}")
                # Clear gpu
                del data_map_reshaped, label_int, label_pred_test, loss, data
                gc.collect()
                torch.cuda.empty_cache()
        print(f"Total Loss: {total_test_loss} Total Accuracy: {total_test_acc/test_count}")
        writer_valid.add_scalar("test_loss", total_test_loss, total_test_step)
        writer_valid_acc.add_scalar("test_acc", total_test_acc/test_count, total_test_step)
        total_test_step = total_test_step + 1
        print("..........Saving the model..........")
        torch.save(model.state_dict(),f"../models/{classification}/{exper_dir}{model_name}/Fold{fold}_Epoch{i}.pt") 
#         if total_test_loss < min_test_loss:
#             min_test_loss = total_test_loss
#             print("..........Saving the model..........")
#             torch.save(model.state_dict(),f"../model/{exper_dir}/Fold{fold}_Epoch{i}.pt")

D:\Anaconda\envs\pytorch\lib\site-packages\ipykernel_launcher.py:92: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/5 [00:00<?, ?it/s]

../data/eegmap_split/rest\train\hc\rest_hc_train_0.pt
../data/eegmap_split/rest\train\hc\rest_hc_train_0_hc_doc_label.pt
../data/eegmap_split/rest\test\hc\rest_hc_test_0.pt
../data/eegmap_split/rest\test\hc\rest_hc_test_0_hc_doc_label.pt
../data/eegmap_split/rest\train\mcs\rest_mcs_train_0.pt
../data/eegmap_split/rest\train\mcs\rest_mcs_train_0_hc_doc_label.pt
../data/eegmap_split/rest\test\mcs\rest_mcs_test_0.pt
../data/eegmap_split/rest\test\mcs\rest_mcs_test_0_hc_doc_label.pt
../data/eegmap_split/rest\train\uws\rest_uws_train_0.pt
../data/eegmap_split/rest\train\uws\rest_uws_train_0_hc_doc_label.pt
../data/eegmap_split/rest\test\uws\rest_uws_test_0.pt
../data/eegmap_split/rest\test\uws\rest_uws_test_0_hc_doc_label.pt
torch.Size([2221, 2400, 10, 11])
torch.Size([2221])
torch.Size([554, 2400, 10, 11])
torch.Size([554])


D:\Anaconda\envs\pytorch\lib\site-packages\ipykernel_launcher.py:155: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/50 [00:00<?, ?it/s]

...

========= Epoch 44 Testing =========
Loss: 0.004813334438949823 Accuracy: 1.0
Loss: 0.009251981973648071 Accuracy: 1.0
Loss: 0.07058294862508774 Accuracy: 1.0
Loss: 0.07082207500934601 Accuracy: 1.0
Loss: 0.09392115473747253 Accuracy: 1.0
Total Loss: 1.6190319061279297 Total Accuracy: 0.9981949329376221
..........Saving the model..........
========= Epoch 45 Training =========
tensor([[-14.9745,  11.5138]], device='cuda:0', grad_fn=<AddmmBackward0>)
Train time: 18750.989167928696
Train steps: 100000, Loss: 0.0
tensor([[-14.2477,  10.8706]], device='cuda:0', grad_fn=<AddmmBackward0>)
Train time: 18914.57305574417
Train steps: 101000, Loss: 0.0
tensor([[-14.9231,  11.4221]], device='cuda:0', grad_fn=<AddmmBackward0>)
Train time: 19077.23112845421
Train steps: 102000, Loss: 0.0
========= Epoch 45 Testing =========
Loss: 45.12818145751953 Accuracy: 0.9300000071525574
Loss: 128.7610321044922 Accuracy: 0.9199999570846558
Loss: 159.46432495117188 Accuracy: 0.9266666769981384
Loss: 219.3265075

  0%|          | 0/50 [00:00<?, ?it/s]

========= Epoch 0 Training =========
tensor([[-0.9000,  0.9428]], device='cuda:0', grad_fn=<AddmmBackward0>)
Train time: 160.10561537742615
Train steps: 1000, Loss: 0.14701777696609497
tensor([[-1.2799,  1.3569]], device='cuda:0', grad_fn=<AddmmBackward0>)
Train time: 322.3889183998108
Train steps: 2000, Loss: 0.06914069503545761
========= Epoch 0 Testing =========
Loss: 37.95658874511719 Accuracy: 0.8700000047683716
Loss: 72.67112731933594 Accuracy: 0.8700000047683716
Loss: 100.08179473876953 Accuracy: 0.8700000047683716
Loss: 156.12167358398438 Accuracy: 0.8524999618530273
Loss: 190.82017517089844 Accuracy: 0.8560000658035278
Total Loss: 209.2842254638672 Total Accuracy: 0.8574007153511047
..........Saving the model..........
========= Epoch 1 Training =========
tensor([[-2.5486,  2.5836]], device='cuda:0', grad_fn=<AddmmBackward0>)
Train time: 549.6409928798676
Train steps: 3000, Loss: 0.005886124912649393
tensor([[-0.0215, -0.0177]], device='cuda:0', grad_fn=<AddmmBackward0>)
Train

Loss: 21.31199073791504 Accuracy: 0.9824999570846558
Loss: 34.128501892089844 Accuracy: 0.9740000367164612
Total Loss: 40.025611877441406 Total Accuracy: 0.9729241728782654
..........Saving the model..........
========= Epoch 11 Training =========
tensor([[ 3.6772, -3.7718]], device='cuda:0', grad_fn=<AddmmBackward0>)
Train time: 3299.878484725952
Train steps: 25000, Loss: 0.0005818104837089777
tensor([[-2.1631,  2.3989]], device='cuda:0', grad_fn=<AddmmBackward0>)
Train time: 3392.4899187088013
Train steps: 26000, Loss: 0.010386461392045021
========= Epoch 11 Testing =========
Loss: 5.908524990081787 Accuracy: 0.9799999594688416
Loss: 14.150157928466797 Accuracy: 0.9649999737739563
Loss: 21.831331253051758 Accuracy: 0.9633333683013916
Loss: 43.45424270629883 Accuracy: 0.9524999856948853
Loss: 57.09932327270508 Accuracy: 0.9440000653266907
Total Loss: 60.79727554321289 Total Accuracy: 0.9458483457565308
..........Saving the model..........
========= Epoch 12 Training =========
tensor([

Loss: 15.17070198059082 Accuracy: 0.9899999499320984
Loss: 46.92131042480469 Accuracy: 0.9699999690055847
Loss: 61.476226806640625 Accuracy: 0.9733333587646484
Loss: 106.34644317626953 Accuracy: 0.9699999690055847
Loss: 130.956298828125 Accuracy: 0.9720000624656677
Total Loss: 130.9609375 Total Accuracy: 0.9747292399406433
..........Saving the model..........
========= Epoch 22 Training =========
tensor([[-10.3987,   9.2855]], device='cuda:0', grad_fn=<AddmmBackward0>)
Train time: 6165.712842464447
Train steps: 49000, Loss: 0.0
tensor([[-10.7970,   9.5500]], device='cuda:0', grad_fn=<AddmmBackward0>)
Train time: 6322.699383735657
Train steps: 50000, Loss: 0.0
tensor([[-11.0564,   9.6845]], device='cuda:0', grad_fn=<AddmmBackward0>)
Train time: 6478.04647898674
Train steps: 51000, Loss: 0.0
========= Epoch 22 Testing =========
Loss: 23.283926010131836 Accuracy: 0.9599999785423279
Loss: 68.92411041259766 Accuracy: 0.9599999785423279
Loss: 69.84127807617188 Accuracy: 0.9700000286102295
Lo

In [7]:
for i in range(1,5):
    print(i)

1
2
3
4
